<a href="https://colab.research.google.com/github/urieliram/Pyomo_example/blob/main/UCP_Pyomo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejemplo básico de uso de Pyomo para resolver un problema de asignación de unidades básico

In [92]:
!pip install -q pyomo # instalación del lenguaje algebraico pyomo
!apt-get install -y -qq glpk-utils #instalación solver glpk
!apt-get install -y -qq coinor-cbc #instalación solver cbc

**Conjuntos**
-  $T$ : Conjunto de intervalos de tiempo.
-  $J$ : Conjunto de unidades generadoras.

**Parámetros**
- $c_j$ : Costo de operación de la unidad generadora \( $j$ \).
-  $c_j^U$ : Costo de  arranque de la unidad generadora \( $j$ \).
-  $De_t$ : Demanda en el intervalo de tiempo \( $t$ \).
-  $R_t$ : Reserva rodante requerida en el intervalo de tiempo \( $t$ \).
-  $R_j^U$ : Límite superior de rampa para la unidad generadora \( $j$ \).
-  $S_j^U$ : Límite superior de rampa para la unidad generadora \( $j$ \).
-  $R_j^D$ : Límite inferior de rampa para la unidad generadora \( $j$ \).
-  $S_j^D$ : Límite inferior de rampa para la unidad generadora \( $j$ \).
- $T_j^U $: Tiempo mínimo encendido para la unidad generadora \( $j$ \) (para $y_{jt}$).
-  $T_j^D$: Tiempo mínimo apagado para la unidad generadora \( $j$ \) (para $z_{jt}$).
-  $P_j^\text{min}$ : Capacidad mínima de generación para la unidad generadora \( $j$ \).
-  $P_j^\text{max}$ : Capacidad máxima de generación para la unidad generadora \( $j$ \).
-  $U_j$ : Periodos encendido la unidad generadora \( $j$ \).
-  $D_j$ : Periodo apagado para la unidad generadora \( $j$ \).

**Variables de decisión**
-  $p_{jt}$ : Generación de la unidad \( $j$ \) en el intervalo \( $t$ \).
-  $\bar{p}_{jt}$ : Generación adicional de la unidad \( $j$ \) en el intervalo \( $t$ \).
-  $v_{jt}$ : Variable binaria que indica si la unidad generadora \( $j$ \) está en operación en el intervalo \( $t$ \).
-  $y_{jt}$ : Variable binaria que indica si se utiliza la unidad generadora \( $j$ \) en el intervalo \( $t$ \).
-  $z_{jt}$ : Variable binaria que indica si se apaga la unidad generadora \( $j$ \) en el intervalo \( $t$ \).



**Modelo**
\begin{align*}
\text{Min} \quad & \sum_{t\in T} \sum_{j\in J} ( c_{j} p_{jt} + c_{j}^\text{U} y_{jt}) \\
\text{Sujeto a:} \quad & \sum_{j\in J} p_{jt} = De_{t}, & t \in T \\
& \sum_{j\in J} \bar{p}_{jt} \geq De_{t} + R_{t}, &  t \in T\\
& v_{j,t-1} - v_{jt} + y_{jt} - z_{jt} = 0 & j\in J, t \in T  \\
& p_{jt} - p_{j,t-1} \leq R_{j}^\text{U} v_{j,t-1} + S_{j}^\text{U} y_{jt} & j\in J, t \in T  \\
& p_{j,t-1} - p_{jt} \leq R_{j}^\text{D} v_{j,t} + S_{j}^\text{D} z_{jt} & j\in J, t \in T  \\
& \sum_{k=t-T_{j}^{U}+1,k \geq 1}^{t} y_{jk} \leq v_{jt} & j\in J, t\in \{L_{j}+1,...,|T|\} \\
& v_{jt}+\sum_{k=t-T_{j}^{D}+1,k \geq 1}^{t} z_{jk} \leq 1 & j\in J, t\in \{F_{j}+1,...,|T|\} \\
& P_{j}^{\text{min}} v_{jt} \leq p_{jt} \leq \bar{p}_{jt} \leq P_{j}^{\text{max}} v_{jt} & j\in J, t \in T  \\
& \bar{p}_{jt} \leq p_{j,t-1} + R_{j}^{\text{U}} v_{j,t-1} + S_{j}^{\text{U}} y_{jt} & j\in J, t \in T  \\
& \bar{p}_{jt} \leq P_{j}^{\text{max}} (v_{jt}-z_{j,t+1})+ z_{j,t+1} S_{j}^{\text{D}} & j\in J, t =\{1,2,...,|T|-1\} \\
& P_{jt}, \bar{jt} \geq 0 \\
& v_{jt}, y_{jt}, z_{jt} = \{0,1\} \\
 \quad \text{donde:}  \\
& L_{j} = min(U_{j},|T|) \\
& F_{j} = min(D_{j},|T|)
\end{align*}

Modelo extraido de: M. F. Anjos and A. J. Conejo., Unit Commitment in Electric Energy Systems. Foundations and Trends in Electric Energy Systems. Vol. 1, No. 4, 220--310, 2017.

In [93]:
#Conjuntos
T    = {1,2,3,4,5,6}
G    = {1,2,3}

# Parámetros
De   = {1: 240, 2: 250, 3: 200, 4: 170, 5: 230, 6: 190}
R    = {1: 0, 2: 10, 3: 10, 4: 10, 5: 10, 6: 10}
Pmax = {1: 300, 2: 200, 3: 100}
Pmin = {1: 80, 2: 50, 3: 30}
cU   = {1: 800, 2: 500, 3: 250}
c    = {1: 5, 2: 15, 3: 30}
RD   = {1: 30, 2: 40, 3: 50}
RU   = {1: 50, 2: 60, 3: 70}
SU   = {1: 100, 2: 70, 3: 40}
SD   = {1: 80, 2: 50, 3: 30}
TU   = {1: 3, 2: 2, 3: 1}
TD   = {1: 2, 2: 2, 3: 2}
U    = {1: 2, 2: 0, 3: 0}
D    = {1: 0, 2: 0, 3: 0}
# Estados iniciales de las unidades generadoras
v0   = {1: 1, 2: 0, 3: 0}
p0   = {1: 120, 2: 0, 3: 0}

In [94]:
# importaciones de librerias
from pyomo.environ import *

# Aqui se define el modelo
def construir_modelo(LP = False, commit=None):
    # Crear un modelo
    model      = ConcreteModel()
    # Conjuntos
    model.G    = Set(initialize = G)
    model.T    = Set(initialize = T)

    # Parametros
    model.Pmax  = Param(model.G , initialize = Pmax , within = Any)
    model.Pmin  = Param(model.G , initialize = Pmin , within = Any)
    model.c     = Param(model.G , initialize = c    , within = Any)
    model.cU    = Param(model.G , initialize = cU   , within = Any)
    model.De    = Param(model.T , initialize = De   , within = Any)
    model.R     = Param(model.T , initialize = R    , within = Any)
    model.RU    = Param(model.G , initialize = RU   , within = Any)
    model.RD    = Param(model.G , initialize = RD   , within = Any)
    model.SU    = Param(model.G , initialize = SU   , within = Any)
    model.SD    = Param(model.G , initialize = SD   , within = Any)
    model.U     = Param(model.G , initialize = U    , within = Any)
    model.D     = Param(model.G , initialize = D    , within = Any)
    model.TU    = Param(model.G , initialize = TU   , within = Any)
    model.TD    = Param(model.G , initialize = TD   , within = Any)
    model.v0    = Param(model.G , initialize = v0   , within = Any)
    model.p0    = Param(model.G , initialize = p0   , within = Any)

    # Variables
    model.p     = Var( model.G , model.T , bounds = (0.0,99999.0) )
    model.pb    = Var( model.G , model.T , bounds = (0.0,99999.0))

    if LP == True:
        model.v     = Var( model.G , model.T , within = UnitInterval)
        model.y     = Var( model.G , model.T , within = UnitInterval)
        model.z     = Var( model.G , model.T , within = UnitInterval)
    else:
        model.v     = Var( model.G , model.T , within = Binary)
        model.y     = Var( model.G , model.T , within = Binary)
        model.z     = Var( model.G , model.T , within = Binary)

    def obj_rule(m):
        return sum(( m.c[j] * m.p[j,t] + m.cU[j] * m.y[j,t] ) for t in m.T for j in m.G)
    model.obj = Objective(rule = obj_rule,sense=minimize)

    def demanda(m,t):
        return sum( m.p[j,t] for j in m.G ) == m.De[t]
    model.demanda = Constraint(model.T, rule = demanda)

    def demanda_reserva(m,t):
        return sum( m.pb[j,t] for j in m.G ) >= m.De[t] + m.R[t]
    model.demanda_reserva = Constraint(model.T, rule = demanda_reserva)

    def logical_rule(m,j,t):
        if t == 1:
            return m.v0[j]    - m.v[j,t] + m.y[j,t] - m.z[j,t] == 0
        else:
            return m.v[j,t-1] - m.v[j,t] + m.y[j,t] - m.z[j,t] == 0
    model.logical_rule = Constraint(model.G,model.T,rule = logical_rule)

    def rampas_subida(m,j,t):
        if t == 1:
            return m.p[j,t] - m.p0[j]    <= m.RU[j] * m.v0[j]    + m.SU[j] * m.y[j,t]
        else:
            return m.p[j,t] - m.p[j,t-1] <= m.RU[j] * m.v[j,t-1] + m.SU[j] * m.y[j,t]
    model.rampas_subida = Constraint(model.G,model.T,rule = rampas_subida)

    def rampas_bajada(m,j,t):
        if t == 1:
            return m.p0[j]    - m.p[j,t] <= m.RD[j] * m.v[j,t] + m.SD[j] * m.z[j,t]
        else:
            return m.p[j,t-1] - m.p[j,t] <= m.RD[j] * m.v[j,t] + m.SD[j] * m.z[j,t]
    model.rampas_bajada = Constraint(model.G,model.T,rule = rampas_bajada)

    def tiempo_min_encendido(m,j,t):
        Lj = min( m.U[j],len(m.T) )
        if t >= Lj + 1:
            return sum( m.y[j,k] for k in range(t-m.TU[j]+1,t) if k>=1) <= m.v[ j , t ]
        else:
            return Constraint.Skip
    model.tiempo_min_encendido = Constraint(model.G,model.T,rule = tiempo_min_encendido)

    def tiempo_min_apagado(m,j,t):
        Fj = min( m.D[j],len(m.T) )
        if t >= Fj + 1:
            return m.v[j,t] + sum( m.z[j,k] for k in range(t-m.TD[j]+1,t) if k>=1) <= 1
        else:
            return Constraint.Skip
    model.tiempo_min_apagado = Constraint(model.G,model.T,rule = tiempo_min_apagado)

    def limite_gen_uno(m,j,t):
        return m.Pmin[j] * m.v[j,t] <= m.p[j,t]
    model.limite_gen_uno = Constraint(model.G,model.T,rule = limite_gen_uno)

    def limite_gen_dos(m,j,t):
        return m.p[j,t] <= m.pb[j,t]
    model.limite_gen_dos = Constraint(model.G,model.T,rule = limite_gen_dos)

    def Limite_gen_tres(m,j,t):
        return m.pb[j,t] <= m.Pmax[j] * m.v[j,t]
    model.Limite_gen_tres = Constraint(model.G,model.T,rule = Limite_gen_tres)

    def limite_arranque(m,j,t):
        if t == 1:
            return m.pb[j,t] <= m.p0[j]    + m.RU[j] * m.v0[j]    + m.SU[j] * m.y[j,t]
        else:
            return m.pb[j,t] <= m.p[j,t-1] + m.RU[j] * m.v[j,t-1] + m.SU[j] * m.y[j,t]
    model.limite_arranque = Constraint(model.G,model.T,rule = limite_arranque)

    def limite_apagado(m,j,t):
        if t == len(m.T):
            return Constraint.Skip
        else:
            return m.pb[j,t] <= m.Pmax[j] * ( m.v[j,t] - m.z[j,t+1] ) + m.z[j,t+1] * m.SD[j]
    model.limite_apagado = Constraint(model.G,model.T,rule = limite_apagado)

    if commit:
      print("FIJANDO SOLUCIÓN ENTERA")
      for key, value in commit.items():
        g, t = key
        model.v[g, t].fix(value) ## Hard fixing

    return model

# Visualización del modelo

In [95]:
# Aqui se instancia el modelo
model = construir_modelo()
## Se imprime el modelo
model.pprint()

17 Set Declarations
    G : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    Limite_gen_tres_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    G*T :   18 : {(1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6)}
    T : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {1, 2, 3, 4, 5, 6}
    limite_apagado_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    G*T :   18 : {(1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6)}
    limite_arranque_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Memb

## Solución del modelo con el solver cbc

In [96]:
SolverFactory('cbc', executable='/usr/bin/cbc').solve(model).write()
print('\n z = ', model.obj())

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 9100.0
  Upper bound: 9100.0
  Number of objectives: 1
  Number of constraints: 169
  Number of variables: 84
  Number of binary variables: 54
  Number of integer variables: 54
  Number of nonzeros: 30
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.08
  Wallclock time: 0.15
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bo

## Solución del modelo con el solver glpk

In [97]:
model = construir_modelo()
SolverFactory('glpk', executable='/usr/bin/glpsol').solve(model) #.write()
print('\n z = ', model.obj())


 z =  9100.0


Visualizando la solución

In [98]:
def imprime_tabla():
  from prettytable import PrettyTable
  # Crear una tabla
  tabla = PrettyTable()
  tabla.field_names = ['G', 'T', 'p', 'pb', 'v', 'y', 'z']

  # Llenar la tabla con los valores de las variables
  for g in G:
      for t in T:
          tabla.add_row([g, t, model.p[g, t].value, model.pb[g, t].value,
                        model.v[g, t].value, model.y[g, t].value, model.z[g, t].value])

  # Imprimir la tabla
  print(tabla)

In [99]:
imprime_tabla()

+---+---+---------------------+---------------------+-----+-----+-----+
| G | T |          p          |          pb         |  v  |  y  |  z  |
+---+---+---------------------+---------------------+-----+-----+-----+
| 1 | 1 |        170.0        |        170.0        | 1.0 | 0.0 | 0.0 |
| 1 | 2 |        200.0        |        210.0        | 1.0 | 0.0 | 0.0 |
| 1 | 3 |        200.0        |        210.0        | 1.0 | 0.0 | 0.0 |
| 1 | 4 |        170.0        |        250.0        | 1.0 | 0.0 | 0.0 |
| 1 | 5 |        200.0        |        220.0        | 1.0 | 0.0 | 0.0 |
| 1 | 6 |        190.0        |        200.0        | 1.0 | 0.0 | 0.0 |
| 2 | 1 |         70.0        |         70.0        | 1.0 | 1.0 | 0.0 |
| 2 | 2 |         50.0        |         50.0        | 1.0 | 0.0 | 0.0 |
| 2 | 3 | 1.4210854715202e-14 | 1.4210854715202e-14 | 0.0 | 0.0 | 1.0 |
| 2 | 4 |         0.0         |         0.0         | 0.0 | 0.0 | 0.0 |
| 2 | 5 |         0.0         |         0.0         | 0.0 | 0.0 

## Fijando una solución y resolviendo

In [100]:
# Llenamos un diccionario con los valores de las variables de asignación
commit = {}
for g in G:
    for t in T:
        commit[(g, t)] = model.v[g, t].value

In [101]:
model = construir_modelo(LP=True,commit=commit)
SolverFactory('glpk', executable='/usr/bin/glpsol').solve(model) #.write()
print('\n z = ', model.obj())

FIJANDO SOLUCIÓN ENTERA

 z =  9100.0


In [102]:
imprime_tabla()

+---+---+-------+-------+-----+-----+------+
| G | T |   p   |   pb  |  v  |  y  |  z   |
+---+---+-------+-------+-----+-----+------+
| 1 | 1 | 170.0 | 170.0 | 1.0 | 0.0 | 0.0  |
| 1 | 2 | 200.0 | 210.0 | 1.0 | 0.0 | 0.0  |
| 1 | 3 | 200.0 | 210.0 | 1.0 | 0.0 | 0.0  |
| 1 | 4 | 170.0 | 180.0 | 1.0 | 0.0 | 0.0  |
| 1 | 5 | 200.0 | 210.0 | 1.0 | 0.0 | 0.0  |
| 1 | 6 | 190.0 | 200.0 | 1.0 | 0.0 | 0.0  |
| 2 | 1 |  70.0 |  70.0 | 1.0 | 1.0 | 0.0  |
| 2 | 2 |  50.0 |  50.0 | 1.0 | 0.0 | 0.0  |
| 2 | 3 |  0.0  |  0.0  | 0.0 | 0.0 | 1.0  |
| 2 | 4 |  0.0  |  0.0  | 0.0 | 0.0 | 0.0  |
| 2 | 5 |  0.0  |  0.0  | 0.0 | 0.0 | 0.0  |
| 2 | 6 |  0.0  |  0.0  | 0.0 | 0.0 | 0.0  |
| 3 | 1 |  0.0  |  0.0  | 0.0 | 0.0 | -0.0 |
| 3 | 2 |  0.0  |  0.0  | 0.0 | 0.0 | 0.0  |
| 3 | 3 |  0.0  |  0.0  | 0.0 | 0.0 | 0.0  |
| 3 | 4 |  0.0  |  0.0  | 0.0 | 0.0 | 0.0  |
| 3 | 5 |  30.0 |  30.0 | 1.0 | 1.0 | 0.0  |
| 3 | 6 |  0.0  |  0.0  | 0.0 | 0.0 | 1.0  |
+---+---+-------+-------+-----+-----+------+
